In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_palette("GnBu_d")
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random 
import timeit
from haversine import haversine # conda install -c conda-forge haversine ''

In [2]:
BRM = pd.read_csv("resources/bremen.csv")
FRB = pd.read_csv("resources/freiburg.csv")

In [3]:
BRM_origin = (BRM["orig_lat"]),(BRM["orig_lng"])
BRM_destination = (BRM["dest_lat"]),(BRM["dest_lng"])

FRB_origin = (FRB["orig_lat"]),(FRB["orig_lng"])
FRB_destination = (FRB["dest_lat"]),(FRB["dest_lng"])

In [4]:
BRM_distance = []
for row in range(len(BRM)):
    BRM_distance.append(haversine(
        (BRM_origin[0][row],BRM_origin[1][row]),(BRM_destination[0][row],BRM_destination[1][row])
    ))
    
BRM["distance"] = BRM_distance

In [5]:
FRB_distance = []
for row in range(len(FRB)):
    FRB_distance.append(haversine(
        (FRB_origin[0][row],FRB_origin[1][row]),(FRB_destination[0][row],FRB_destination[1][row])
    ))
    
FRB["distance"] = FRB_distance


In [6]:
BRM.tail(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
157573,2019-11-18,09:23:00,20999,bremen,0 days 00:09:00.000000000,53.070369,8.821749,53.079360,8.813848,1.130559
157574,2019-11-19,15:35:00,20999,bremen,0 days 00:43:00.000000000,53.079591,8.813925,53.145093,8.910599,9.730569
157575,2019-11-21,17:45:00,20999,bremen,0 days 00:11:00.000000000,53.145798,8.909947,53.158160,8.945250,2.726051


In [7]:
FRB.tail(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
127526,2020-01-20,10:03:00,32999,freiburg,0 days 00:24:00.000000000,48.000858,7.849587,48.002664,7.851253,0.235992
127527,2020-01-20,13:55:00,32999,freiburg,0 days 00:07:00.000000000,48.002664,7.851253,47.997430,7.842500,0.873416
127528,2020-01-20,14:16:00,32999,freiburg,0 days 00:18:00.000000000,47.997430,7.842500,48.013770,7.807080,3.200782


We cheched the longest trip duration both for the city of Bremen and the city of Freiburg. By doing so we came to the conclusion that both durations are just under 3 hours. So by removing the day-values and the redundant milliseconds we are making the data easier to grasp for the average reader and easier to work with for the data scientist.

In [8]:
 BRM["trip_duration"].max()

'0 days 02:59:00.000000000'

In [9]:
 FRB["trip_duration"].max()

'0 days 02:59:00.000000000'

In [10]:
BRM["trip_duration"] = BRM["trip_duration"].astype(str).str.extract("days (.*?)\.")

In [11]:
FRB["trip_duration"] = FRB["trip_duration"].astype(str).str.extract("days (.*?)\.")

In [12]:
FRB.head(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
0,2019-05-06,14:22:00,32560,freiburg,00:07:00,47.993178,7.795708,47.994027,7.796084,0.098454
1,2019-05-07,10:42:00,32560,freiburg,00:07:00,47.994191,7.796853,47.991960,7.797405,0.251462
2,2019-05-07,11:02:00,32560,freiburg,00:09:00,47.992044,7.797352,47.992000,7.797478,0.010643


In [13]:
BRM.head(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
0,2019-04-21,17:34:00,20507,bremen,00:13:00,53.078923,8.884911,53.078711,8.899906,1.001900
1,2019-05-23,15:33:00,20507,bremen,00:07:00,53.078923,8.884911,53.078004,8.876828,0.549476
2,2019-06-14,22:30:00,20507,bremen,00:08:00,53.078923,8.884911,53.074731,8.876003,0.755886


As part of the data cleaning we are making the assumption that all the trips with total distance under 100m are due to some kind of error, such as unintended log-in, hence they are removed from the data set.  

In [14]:
#dropping the noise-rows
indexF = FRB[(FRB["distance"] < 0.1)].index
FRB.drop(indexF, inplace=True)
indexB = BRM[(BRM["distance"] < 0.1)].index
BRM.drop(indexB, inplace=True)

#resetting the indices after the cleaning
FRB=FRB.reset_index(drop=True)
BRM=BRM.reset_index(drop=True)

In [15]:
FRB

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
0,2019-05-07,10:42:00,32560,freiburg,00:07:00,47.994191,7.796853,47.991960,7.797405,0.251462
1,2019-05-07,12:48:00,32560,freiburg,00:23:00,47.992107,7.797499,47.994271,7.796668,0.248495
2,2019-05-08,09:00:00,32560,freiburg,00:03:00,47.994178,7.796721,47.992307,7.796708,0.208048
3,2019-05-08,11:33:00,32560,freiburg,00:04:00,47.992307,7.796708,47.993244,7.796573,0.104724
4,2019-05-08,11:38:00,32560,freiburg,00:03:00,47.993244,7.796575,47.992960,7.794476,0.159398
...,...,...,...,...,...,...,...,...,...,...
126965,2020-01-20,08:14:00,32999,freiburg,00:12:00,48.012050,7.854987,47.994729,7.846862,2.018664
126966,2020-01-20,09:10:00,32999,freiburg,00:04:00,47.996100,7.846160,48.000858,7.849587,0.587308
126967,2020-01-20,10:03:00,32999,freiburg,00:24:00,48.000858,7.849587,48.002664,7.851253,0.235992
126968,2020-01-20,13:55:00,32999,freiburg,00:07:00,48.002664,7.851253,47.997430,7.842500,0.873416


Another assumption made for the purpose of data cleaning is that trips with average speed under 0.5kmh are also of no interest for us.

In [20]:
#creating helper-columns for the calculation of the  average kmh
FRB["trip_duration_seconds"] = pd.to_timedelta(FRB['trip_duration'])
FRB["trip_duration_seconds"] = FRB["trip_duration_seconds"].dt.seconds
FRB["trip_duration_hours"] = (FRB["trip_duration_seconds"]/3600)
FRB["kmh"] = (FRB["distance"] / FRB["trip_duration_hours"])

BRM["trip_duration_seconds"] = pd.to_timedelta(BRM['trip_duration'])
BRM["trip_duration_seconds"] = BRM["trip_duration_seconds"].dt.seconds
BRM["trip_duration_hours"] = (BRM["trip_duration_seconds"]/3600)
BRM["kmh"] = (BRM["distance"] / BRM["trip_duration_hours"])

# applying the condition
indexF = FRB[(FRB["kmh"] < 5)].index
FRB.drop(indexF, inplace=True) 
indexB = BRM[(BRM["kmh"] < 5)].index
BRM.drop(indexB, inplace=True) 

#resetting the indices after the cleaning
FRB=FRB.reset_index(drop=True)
BRM=BRM.reset_index(drop=True)

#removing the helper-columns from the data set
FRB = FRB.drop("trip_duration_seconds", 1)
FRB = FRB.drop("trip_duration_hours", 1)
FRB = FRB.drop("kmh", 1)

BRM = BRM.drop("trip_duration_seconds", 1)
BRM = BRM.drop("trip_duration_hours", 1)
BRM = BRM.drop("kmh", 1)



In [21]:
FRB

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
0,2019-07-01,19:51:00,32560,freiburg,00:07:00,48.002664,7.851253,48.008110,7.850470,0.608364
1,2019-07-02,22:14:00,32560,freiburg,00:10:00,47.984584,7.840984,47.997669,7.852481,1.687895
2,2019-07-03,05:30:00,32560,freiburg,00:06:00,47.997669,7.852481,47.997564,7.844473,0.595944
3,2019-07-03,12:06:00,32560,freiburg,00:12:00,47.980291,7.821829,47.989120,7.847720,2.162640
4,2019-07-04,06:36:00,32560,freiburg,00:17:00,47.991531,7.879938,47.984584,7.840984,3.000158
...,...,...,...,...,...,...,...,...,...,...
100875,2020-01-19,14:38:00,32999,freiburg,00:16:00,47.988072,7.873674,48.012050,7.854987,3.006990
100876,2020-01-20,08:14:00,32999,freiburg,00:12:00,48.012050,7.854987,47.994729,7.846862,2.018664
100877,2020-01-20,09:10:00,32999,freiburg,00:04:00,47.996100,7.846160,48.000858,7.849587,0.587308
100878,2020-01-20,13:55:00,32999,freiburg,00:07:00,48.002664,7.851253,47.997430,7.842500,0.873416


In [17]:
BRM.tail(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
148511,2019-11-18,09:23:00,20999,bremen,00:09:00,53.070369,8.821749,53.079360,8.813848,1.130559
148512,2019-11-19,15:35:00,20999,bremen,00:43:00,53.079591,8.813925,53.145093,8.910599,9.730569
148513,2019-11-21,17:45:00,20999,bremen,00:11:00,53.145798,8.909947,53.158160,8.945250,2.726051


In [18]:
FRB.tail(3)

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,distance
124760,2020-01-20,10:03:00,32999,freiburg,00:24:00,48.000858,7.849587,48.002664,7.851253,0.235992
124761,2020-01-20,13:55:00,32999,freiburg,00:07:00,48.002664,7.851253,47.997430,7.842500,0.873416
124762,2020-01-20,14:16:00,32999,freiburg,00:18:00,47.997430,7.842500,48.013770,7.807080,3.200782


As a conclusion:
the data points in the city of Bremen have been reduced from 157 576 (before the cleaning) to 148 514 (after the cleaning);
the data points in the city of Freiburg have been reduced from 127 529 (before the cleaning) to 124 763 (after the cleaning);